# Segmental retention models for representing the hydraulic properties of evolving structured soils

This notebook recreates Fig. 4.

In [ ]:
import segmental_models as svg
import vangenuchten as vg
import pandas as pd

import numpy as np
from scipy.constants import micro, milli, nano, centi
import matplotlib.pyplot as plt

In [ ]:
threshold_h = np.array([600 * centi, 0.3])

posd_h = np.concatenate((threshold_h, [0]))

Global settings for plots

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

h = np.logspace(-2.5, 2)

# This is the h over which conductivity will be integrated. It must be a larger range than what is plotted.
# Specifically, the high end must be high enough so that dmc/dh goes pretty much to zero *for the (de)compaction that
# you're making*. The low end should be the same as for h
h_int = np.logspace(-2.5, 5)

plotlim_h = np.flatnonzero(h_int > np.amax(h))[0]

## Decompaction

In [ ]:
soil = {'n': 1.073, 'a': 0.044 / centi, 'mcr': 0.0, 'mcs': 0.407, 'Ks': 14.064557351537498 * centi / 60 * 60 * 24}

M = soil['Ks'] / soil['a'] ** 2 / (soil['mcs'] - soil['mcr']) ** 2.5

# Initial Van Genuchten retention and conductivity. Dimensions: [h]
VG_mc = vg.mc(h, **soil)
VG_K = (
    soil['Ks'] * vg.Kr(vg.S(h_int[None, :], **soil), **soil)[0]
)  # [0] removes the soil type dimension we're not using

# Retention curve segments [soil type, class, h]
sVG_comp = svg.comps(h, threshold_h, **soil)

# Retention sub-curves [class, h]
sVG_mc = svg.mc(sVG_comp, threshold_h, **soil)[0]  # [0] as above

In [ ]:
pd.read_csv('MJB2020_PoSD.csv', index_col=[0, 1], comment='#')

In [ ]:
posd_rel = (
    pd.read_csv('MJB2020_PoSD.csv', index_col=[0, 1], comment='#')
    .drop(1988)
    .groupby('treatment', group_keys=False)
    .apply(lambda da: da / da.loc[1956])
)

posd_rel['phi_mac'] = 1.0  # Macroporosity is constant in the model of Meurer, Chenu, et al. (2020)

posd_rel

In [ ]:
posd = sVG_mc[:, 0].copy()  # Initial porosities (micro, meso, macro)
posd[1] += posd[0]  # Initial porosities (micro, matrix i.e. micro + meso, macro)

# Apply relative porosity changes from Meurer, Chenu, et al. (2020) to our initial pore size distribution
posd = posd * np.vstack(
    (posd_rel.xs('bare', level='treatment').to_numpy(), posd_rel.xs('manure', level='treatment').to_numpy())
)  #  Bare fallow treatment (all years) first, followed by manure (all years)

posd[:, 1] -= posd[:, 0]  # Transform back from (micro, matrix, macro) to (micro, meso, macro)

# Weights or (de)compaction factors [year, component (micro, meso, macro)]
w = posd / posd[0]

In [ ]:
# (De)compacted retention curves [year, h]
sVG_mc_decomp = (sVG_mc[None, ...] * w[..., None]).sum(axis=-2)

In [ ]:
# (De)compacted conductivity curves [year, h]
sVG_K_decomp = (svg.dmcdh_int(h_int, threshold_h, **soil)[None, ...] * w[..., None]).sum(axis=-2) ** 2

sVG_K_decomp *= (
    M
    * (svg.mc(svg.comps(h_int[1:], threshold_h, **soil), threshold_h, **soil)[0, None, ...] * w[..., None]).sum(axis=-2)
    ** 0.5
)

# The (0, None,) part of the slice of above doesn't do anything here, but it signifies an intent: The 0 removes the soil
# type dimension we're not using (it has length 1), and None adds a year dimension we didn't have

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=sVG_mc_decomp.shape[0] // 2, figsize=(11, 6.75), sharex=True, sharey='row')

axes[0, 0].set_xscale('log')
axes[1, 0].set_yscale('log')

for j, ax in enumerate(axes[0]):
    ax.plot(h, VG_mc, 'k', label='original retention / conductivity')

    ax.plot(h, soil['mcr'] + sVG_mc_decomp[j], label='estimated ret. / cond. (bare fallow)')
    ax.plot(
        threshold_h,
        np.cumsum(posd[j, :-1]),
        'o',
        color=colors[0],
        label='simulated porosity (bare fallow)',
        markerfacecolor='none',
        markeredgewidth=1.5,
        zorder=5,
    )

    ax.plot(h, soil['mcr'] + sVG_mc_decomp[j + 3], color=colors[2], label='est. ret. / cond. (manure)')
    ax.plot(
        threshold_h,
        np.cumsum(posd[j + 3, :-1]),
        'o',
        color=colors[2],
        label='simulated porosity (manure)',
        markerfacecolor='none',
        markeredgewidth=1.5,
        zorder=6,
    )

    ax.grid(alpha=0.25)

for j, ax in enumerate(axes[1]):
    ax.plot(h_int[:plotlim_h], VG_K[:plotlim_h], 'k', label='original')
    ax.plot(h_int[:plotlim_h], sVG_K_decomp[j, :plotlim_h], label='bare')
    ax.plot(h_int[:plotlim_h], sVG_K_decomp[j + 3, :plotlim_h], color=colors[2], label='manure')
    ax.grid(alpha=0.25)

for axo, title in zip(axes[0], posd_rel.index.unique(level='year')):
    axo.set_title(title)

axes[0, 0].legend(loc='upper center', ncol=3, bbox_to_anchor=(1.6, -0.02))

axes[0, 0].invert_xaxis()
axes[0, 0].set_ylabel(r'water content $\theta$ [-]')
axes[1, 0].set_ylabel(r'hydr. conductivity $K$ [m/s]')

fig.supxlabel('pressure head $h$ [m]', y= .025)

plt.subplots_adjust(wspace=0.1, hspace=0.3)

if 'xticklabels' in locals():
    axes[1,0].set_xticklabels(xticklabels)

    plt.savefig('MJB.pdf', bbox_inches="tight", pad_inches=.01)

In [ ]:
# Add minus signs to tick labels (replot after running this cell)
xticklabels = axes[1,0].get_xticklabels()

for Text in xticklabels:
    Text.set_text(Text.get_text().replace('10','\N{MINUS SIGN}10'))